In [ ]:
import os
import sys
import json
import numpy as np
from datetime import datetime
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
import pandas as pd
import torchvision.transforms as T
import torchvision.transforms.functional as TF
import random

from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from PIL import Image
from torchvision import models
from typing import Tuple, Optional

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Set random seed for reproducibility
torch.manual_seed(789)
np.random.seed(789)
random.seed(789)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(789)


Using device: cuda


# 1. Model and Trainer

## 1.1 Model Definition

In [ ]:
# Model Implementation - Calorie Prediction Only (No Segmentation)

class ResNetEncoder(nn.Module):
    """ResNet encoder that extracts feature maps"""
    
    def __init__(self, encoder_type: str = 'resnet18', pretrained: bool = False, in_channels: int = 3):
        super().__init__()
        
        # Load appropriate ResNet
        if encoder_type == 'resnet18':
            resnet = models.resnet18(pretrained=pretrained)
            self.out_channels = 512
        elif encoder_type == 'resnet34':
            resnet = models.resnet34(pretrained=pretrained)
            self.out_channels = 512
        else:
            raise ValueError(f"Unsupported encoder: {encoder_type}")
        
        # Modify first conv if we have different input channels (e.g., 1 for depth)
        if in_channels != 3:
            self.conv1 = nn.Conv2d(
                in_channels, 64, kernel_size=7, stride=2, padding=3, bias=False
            )
        else:
            self.conv1 = resnet.conv1
        
        self.bn1 = resnet.bn1
        self.relu = resnet.relu
        self.maxpool = resnet.maxpool
        
        # ResNet layers
        self.layer1 = resnet.layer1  # Output: 64 channels
        self.layer2 = resnet.layer2  # Output: 128 channels
        self.layer3 = resnet.layer3  # Output: 256 channels
        self.layer4 = resnet.layer4  # Output: 512 channels
    
    def forward(self, x):
        """
        Args:
            x: Input tensor (B, C, H, W)
        Returns:
            Feature map (B, 512, H/32, W/32)
        """
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        return x


class MiddleFusionModule(nn.Module):
    """Middle fusion: Concatenate RGB and Depth features, then merge with 1x1 conv"""
    
    def __init__(self, rgb_channels: int = 512, depth_channels: int = 512, output_channels: int = 512):
        super().__init__()
        
        # 1x1 convolution to merge features
        self.fusion_conv = nn.Conv2d(
            rgb_channels + depth_channels,
            output_channels,
            kernel_size=1,
            bias=False
        )
        self.bn = nn.BatchNorm2d(output_channels)
        self.relu = nn.ReLU(inplace=True)
    
    def forward(self, rgb_features, depth_features):
        """
        Args:
            rgb_features: (B, 512, H, W)
            depth_features: (B, 512, H, W)
        Returns:
            Fused features: (B, 512, H, W)
        """
        # Concatenate along channel dimension
        fused = torch.cat([rgb_features, depth_features], dim=1)  # (B, 1024, H, W)
        
        # Apply 1x1 conv to reduce channels
        fused = self.fusion_conv(fused)  # (B, 512, H, W)
        fused = self.bn(fused)
        fused = self.relu(fused)
        
        return fused


class RegressionHead(nn.Module):
    def __init__(self, in_channels: int = 512, dropout_rate: float = 0.4):
        super().__init__()
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_channels, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate),
            nn.Linear(256, 128),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate),
            nn.Linear(128, 1)
        )
    
    def forward(self, x):
        x = self.avgpool(x)  # (B, C, 1, 1)
        x = self.fc_layers(x)  # (B, 1)
        return x

# Fusion factory function
def get_fusion_module(fusion_name, rgb_channels, depth_channels, output_channels):
    """
    Factory function to get fusion module
    
    Args:
        fusion_name: Name of fusion module ('middle' or 'inception')
        rgb_channels: Number of RGB feature channels
        depth_channels: Number of depth feature channels
        output_channels: Number of output channels
    
    Returns:
        Fusion module
    """
    if fusion_name == 'middle':
        return MiddleFusionModule(
            rgb_channels=rgb_channels,
            depth_channels=depth_channels,
            output_channels=output_channels
        )
    else:
        raise ValueError(f"Unknown fusion module: {fusion_name}")

# Modify DualStreamCaloriePredictor to use different fusion modules
class DualStreamCaloriePredictor(nn.Module):
    """
    Dual-stream CNN for calorie prediction using RGB and Depth images
    Architecture: ResNet encoders + Fusion module + Regression head
    """
    
    def __init__(
        self,
        encoder: str = 'resnet18',
        fusion: str = 'middle',
        fusion_channels: int = 512,
        dropout_rate: float = 0.4,
        pretrained: bool = False
    ):
        super().__init__()
        
        # RGB and Depth encoders
        self.rgb_encoder = ResNetEncoder(encoder, pretrained=pretrained, in_channels=3)
        self.depth_encoder = ResNetEncoder(encoder, pretrained=pretrained, in_channels=1)
        
        # Create fusion module based on specified type
        self.fusion = get_fusion_module(
            fusion_name=fusion,
            rgb_channels=self.rgb_encoder.out_channels,
            depth_channels=self.depth_encoder.out_channels,
            output_channels=fusion_channels
        )
        
        # Regression head for calorie prediction
        self.regression_head = RegressionHead(
            in_channels=fusion_channels,
            dropout_rate=dropout_rate
        )
    
    def forward(self, rgb, depth):
        """
        Args:
            rgb: RGB images (B, 3, H, W)
            depth: Depth images (B, 1, H, W)
        
        Returns:
            calorie_pred: Predicted calories (B, 1)
        """
        # Extract features
        rgb_features = self.rgb_encoder(rgb)      # (B, 512, H/32, W/32)
        depth_features = self.depth_encoder(depth)  # (B, 512, H/32, W/32)
        
        # Fuse features
        fused_features = self.fusion(rgb_features, depth_features)  # (B, 512, H/32, W/32)
        
        # Predict calories
        calorie_pred = self.regression_head(fused_features)  # (B, 1)
        
        return calorie_pred
    
    def get_num_parameters(self):
        """Get total number of trainable parameters"""
        return sum(p.numel() for p in self.parameters() if p.requires_grad)

# Updated build_model function
def build_model(encoder='resnet18', fusion='middle', regression_head='standard', 
                pretrained=False, dropout_rate=0.4, fusion_channels=512, **kwargs):
    """
    Factory function to build models
    """
    return DualStreamCaloriePredictor(
        encoder=encoder,
        fusion=fusion,
        fusion_channels=fusion_channels,
        dropout_rate=dropout_rate,
        pretrained=pretrained
    )

## 1.2 Trainer Definition

In [ ]:
import math
def get_warmup_cosine_scheduler(optimizer, warmup_steps, total_steps, min_lr_ratio=0.0):
    def lr_lambda(current_step):
        if current_step < warmup_steps:
            return float(current_step) / float(max(1, warmup_steps))
        else:
            progress = float(current_step - warmup_steps) / float(max(1, total_steps - warmup_steps))
            return min_lr_ratio + (1.0 - min_lr_ratio) * 0.5 * (1.0 + math.cos(math.pi * progress))
    
    return optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

class EarlyStopping:
    """Early stopping to stop training when validation loss stops improving"""
    
    def __init__(self, patience: int = 10, min_delta: float = 0.0, mode: str = 'min'):
        """
        Args:
            patience: Number of epochs with no improvement after which training will be stopped
            min_delta: Minimum change to qualify as an improvement
            mode: 'min' or 'max' - whether lower or higher metric is better
        """
        self.patience = patience
        self.min_delta = min_delta
        self.mode = mode
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.best_epoch = 0
        
    def __call__(self, score, epoch):
        if self.best_score is None:
            self.best_score = score
            self.best_epoch = epoch
            return False
        
        if self.mode == 'min':
            improved = score < (self.best_score - self.min_delta)
        else:
            improved = score > (self.best_score + self.min_delta)
        
        if improved:
            self.best_score = score
            self.best_epoch = epoch
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
                
        return self.early_stop

class Trainer:
    """Training manager for calorie prediction"""
    
    def __init__(
        self,
        model,
        train_loader,
        val_loader,
        criterion,
        optimizer,
        scheduler,
        device,
        output_dir,
        early_stopping_patience=15,
        scheduler_step_on_batch=False
    ):
        self.model = model
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.criterion = criterion
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.device = device
        self.output_dir = output_dir
        self.scheduler_step_on_batch = scheduler_step_on_batch
        
        # Early stopping
        self.early_stopping = EarlyStopping(
            patience=early_stopping_patience,
            min_delta=0.1,
            mode='min'
        )
        
        # Tensorboard
        self.writer = SummaryWriter(log_dir=os.path.join(output_dir, 'tensorboard'))
        
        # Tracking
        self.best_val_loss = float('inf')
        self.train_losses = []
        self.val_losses = []
        self.best_metrics = {}
    
    def train_epoch(self):
        """Train for one epoch"""
        self.model.train()
        total_loss = 0.0
        num_batches = 0
        
        pbar = tqdm(self.train_loader, desc="Training")
        for batch_idx, batch in enumerate(pbar):
            # Move to device
            rgb = batch['rgb'].to(self.device)
            depth = batch['depth'].to(self.device)
            calories = batch['calorie'].to(self.device)
            
            # Forward pass
            self.optimizer.zero_grad()
            calorie_pred = self.model(rgb, depth)
            
            # Compute loss (MSE for calorie prediction)
            loss = self.criterion(calorie_pred.squeeze(), calories)
            
            # Backward pass
            loss.backward()
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
            self.optimizer.step()
            
            # Update learning rate (if step_on_batch)
            if self.scheduler_step_on_batch and self.scheduler:
                self.scheduler.step()
            
            # Track metrics
            total_loss += loss.item()
            num_batches += 1
            
            # Update progress bar
            pbar.set_postfix({'Loss': f'{loss.item():.4f}'})
        
        return total_loss / num_batches
    
    def validate_epoch(self):
        """Validate for one epoch"""
        self.model.eval()
        total_loss = 0.0
        all_predictions = []
        all_targets = []
        
        with torch.no_grad():
            for batch in tqdm(self.val_loader, desc="Validation"):
                # Move to device
                rgb = batch['rgb'].to(self.device)
                depth = batch['depth'].to(self.device)
                calories = batch['calorie'].to(self.device)
                
                # Forward pass
                calorie_pred = self.model(rgb, depth)
                
                # Compute loss
                loss = self.criterion(calorie_pred.squeeze(), calories)
                total_loss += loss.item()
                
                # Store predictions and targets for metrics
                all_predictions.extend(calorie_pred.squeeze().cpu().numpy())
                all_targets.extend(calories.cpu().numpy())
        
        # Calculate metrics
        avg_loss = total_loss / len(self.val_loader)
        predictions = np.array(all_predictions)
        targets = np.array(all_targets)
        
        mae = np.mean(np.abs(predictions - targets))
        
        return avg_loss, mae
    
    def train(self, num_epochs):
        """Full training loop"""
        print(f"Starting training for {num_epochs} epochs...")
        
        for epoch in range(num_epochs):
            print(f"\nEpoch {epoch+1}/{num_epochs}")
            
            # Train
            train_loss = self.train_epoch()
            
            # Validate
            val_loss, mae = self.validate_epoch()
            
            # Update learning rate (if not step_on_batch)
            if not self.scheduler_step_on_batch and self.scheduler:
                self.scheduler.step(val_loss)
            
            # Log metrics
            self.writer.add_scalar('Loss/Train', train_loss, epoch)
            self.writer.add_scalar('Loss/Val', val_loss, epoch)
            self.writer.add_scalar('MAE', mae, epoch)
            
            # Save best model
            if val_loss < self.best_val_loss:
                self.best_val_loss = val_loss
                self.best_metrics = {
                    'epoch': epoch + 1,
                    'val_loss': val_loss,
                    'mae': mae,
                }
                
                # Save model checkpoint
                torch.save({
                    'epoch': epoch + 1,
                    'model_state_dict': self.model.state_dict(),
                    'optimizer_state_dict': self.optimizer.state_dict(),
                    'val_loss': val_loss,
                    'mae': mae,
                }, os.path.join(self.output_dir, 'best_model.pth'))
            
            # Print epoch results
            print(f"Train Loss: {train_loss:.4f}")
            print(f"Val Loss: {val_loss:.4f}")
            print(f"MAE: {mae:.2f}")
            
            # Early stopping
            if self.early_stopping(val_loss, epoch):
                print(f"Early stopping triggered after {epoch+1} epochs")
                print(f"Best epoch: {self.early_stopping.best_epoch+1}")
                break
        
        self.writer.close()
        print(f"\nTraining completed!")
        print(f"Best validation loss: {self.best_val_loss:.4f}")

# 2. Dataset

## 2.1 Dataset Definition

In [ ]:
# Dataset Implementation
class Nutrition5KDataset(Dataset):
    """
    Dataset class for Nutrition5K with multi-modal inputs (RGB + Depth)
    """
    
    def __init__(
        self,
        csv_path: str,
        data_root: str,
        split: str = 'train',
        augment: bool = True,
        img_size: int = 224,
    ):
        self.data_root = data_root
        self.split = split
        self.augment = augment
        self.img_size = img_size
        
        # Load CSV
        self.df = pd.read_csv(csv_path)
        if 'Value' in self.df.columns and 'calories' not in self.df.columns:
            self.df = self.df.rename(columns={'Value': 'calories'})
        if 'calories' not in self.df.columns:
            raise ValueError("CSV file must contain a 'calories' column or a 'Value' column that can be renamed")
        self.df = self.df[self.df['calories'] < 3000].reset_index(drop=True)
                
        self.color_dir = os.path.join(data_root, 'color')
        self.depth_raw_dir = os.path.join(data_root, 'depth_raw')
        
        self.valid_indices = self._validate_dataset()
        print(f"Loaded {len(self.valid_indices)} valid samples out of {len(self.df)}")
        
        # Color normalization (ImageNet stats as baseline)
        self.color_normalize = T.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )
        
    def _validate_dataset(self):
        """This method ensure that the code don't break when there are corrupted images.""""
        valid_indices = []
        
        for idx in range(len(self.df)):
            dish_id = self.df.iloc[idx]['ID']
            
            rgb_path = os.path.join(self.color_dir, dish_id, 'rgb.png')
            depth_path = os.path.join(self.depth_raw_dir, dish_id, 'depth_raw.png')
            
            # Check if files exist
            if not os.path.exists(rgb_path):
                continue
            if not os.path.exists(depth_path):
                continue
            
            # Try to load images to check for corruption
            try:
                with Image.open(rgb_path) as img:
                    img.verify()
                with Image.open(depth_path) as img:
                    img.verify()
                valid_indices.append(idx)
            except Exception as e:
                continue
                
        return valid_indices
    
    def __len__(self):
        return len(self.valid_indices)
    
    def _load_image_safe(self, path: str, mode: str = 'RGB') -> Optional[Image.Image]:
        """Safely load an image with error handling"""
        try:
            with Image.open(path) as img:
                return img.convert(mode).copy()
        except Exception as e:
            return None
    
    def _apply_augmentation(self, rgb_img, depth_img):
        """Apply geometric augmentation only (no color changes)"""
        if not self.augment:
            return rgb_img, depth_img
        
        # Convert to tensors first
        rgb_tensor = TF.to_tensor(rgb_img)
        depth_tensor = TF.to_tensor(depth_img)
        
        # Random horizontal flip
        if random.random() > 0.5:
            rgb_tensor = TF.hflip(rgb_tensor)
            depth_tensor = TF.hflip(depth_tensor)
        
        # Random rotation (±15 degrees)
        if random.random() > 0.5:
            angle = random.uniform(-15, 15)
            rgb_tensor = TF.rotate(rgb_tensor, angle)
            depth_tensor = TF.rotate(depth_tensor, angle)
        
        # Random resized crop
        if random.random() > 0.4:  # 60% probability
            i, j, h, w = T.RandomResizedCrop.get_params(
                rgb_tensor, scale=(0.75, 1.0), ratio=(0.9, 1.1)
            )
            rgb_tensor = TF.resized_crop(rgb_tensor, i, j, h, w, (self.img_size, self.img_size))
            depth_tensor = TF.resized_crop(depth_tensor, i, j, h, w, (self.img_size, self.img_size))
        
        # Convert back to PIL
        rgb_img = TF.to_pil_image(rgb_tensor)
        depth_img = TF.to_pil_image(depth_tensor)
        
        return rgb_img, depth_img
    
    def _resize_and_center_crop(self, img, target_size: int = 256):
        """
        Resize and center crop image to target_size x target_size
        Matches the preprocessing in the Nutrition5k paper
        
        Args:
            img: PIL Image
            target_size: Target size (default 256x256 as per paper)
        
        Returns:
            Cropped PIL Image
        """
        # Get original dimensions
        width, height = img.size
        
        # Resize so the shorter side is target_size
        if width < height:
            new_width = target_size
            new_height = int(target_size * height / width)
        else:
            new_height = target_size
            new_width = int(target_size * width / height)
        
        img = img.resize((new_width, new_height), Image.LANCZOS)
        
        # Center crop to target_size x target_size
        left = (new_width - target_size) // 2
        top = (new_height - target_size) // 2
        right = left + target_size
        bottom = top + target_size
        
        img = img.crop((left, top, right, bottom))
        
        return img
    
    def __getitem__(self, idx):
        """Get a single sample"""
        actual_idx = self.valid_indices[idx]
        row = self.df.iloc[actual_idx]
        
        dish_id = row['ID']
        calorie = float(row['calories'])
        
        # Load images
        rgb_path = os.path.join(self.color_dir, dish_id, 'rgb.png')
        depth_path = os.path.join(self.depth_raw_dir, dish_id, 'depth_raw.png')
        
        rgb_img = self._load_image_safe(rgb_path, 'RGB')
        depth_img = self._load_image_safe(depth_path, 'L')  # Grayscale for depth
        
        # Fallback: return a black image
        if rgb_img is None or depth_img is None:
            rgb_img = Image.new('RGB', (self.img_size, self.img_size), (0, 0, 0))
            depth_img = Image.new('L', (self.img_size, self.img_size), 0)
        
        # Apply augmentation
        rgb_img, depth_img = self._apply_augmentation(rgb_img, depth_img)
        
        # Resize and center crop to match paper preprocessing (256x256)
        rgb_img = self._resize_and_center_crop(rgb_img, target_size=self.img_size)
        depth_img = self._resize_and_center_crop(depth_img, target_size=self.img_size)
        
        # Convert to tensors
        rgb_tensor = TF.to_tensor(rgb_img)  # (3, H, W)
        depth_tensor = TF.to_tensor(depth_img)  # (1, H, W)
        
        # Normalize RGB
        rgb_tensor = self.color_normalize(rgb_tensor)
        
        # Normalize depth (0-1 range, assuming depth is already in reasonable range)
        depth_tensor = depth_tensor / 255.0
        
        return {
            'dish_id': dish_id,
            'rgb': rgb_tensor,
            'depth': depth_tensor,
            'calorie': torch.tensor(calorie, dtype=torch.float32)
        }


def create_train_val_split(csv_path: str, val_ratio: float = 0.15, random_seed: int = 42):
    """
    Create train/validation split CSV files
    """
    # Read original CSV
    df = pd.read_csv(csv_path)    
    
    # Shuffle with fixed seed
    df_shuffled = df.sample(frac=1, random_state=random_seed).reset_index(drop=True)
    
    # Split
    val_size = int(len(df_shuffled) * val_ratio)
    train_df = df_shuffled[val_size:]
    val_df = df_shuffled[:val_size]
    
    # Save temporary CSV files
    base_dir = os.path.dirname(csv_path)
    train_csv = os.path.join(base_dir, 'train_split.csv')
    val_csv = os.path.join(base_dir, 'val_split.csv')
    
    train_df.to_csv(train_csv, index=False)
    val_df.to_csv(val_csv, index=False)
    
    return train_csv, val_csv

## 2.2 Dataset Loading

In [ ]:
# Configuration - Update these paths to match your setup
DATA_ROOT = './Nutrition5K/Nutrition5K/train'  # Path to training data directory
CSV_PATH = './Nutrition5K/Nutrition5K/nutrition5k_train.csv'  # Path to training CSV
OUTPUT_DIR = './experiments'  # Directory to save experiment results

# Global training hyperparameters
BATCH_SIZE = 32
NUM_EPOCHS = 40
VAL_RATIO = 0.15
IMG_SIZE = 256
NUM_WORKERS = 4

print("Configuration:")
print(f"  Data root: {DATA_ROOT}")
print(f"  CSV path: {CSV_PATH}")
print(f"  Output directory: {OUTPUT_DIR}")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Number of epochs: {NUM_EPOCHS}")
print(f"  Image size: {IMG_SIZE}")
print(f"  Workers: {NUM_WORKERS}")

# Create output directory
os.makedirs(OUTPUT_DIR, exist_ok=True)


Configuration:
  Data root: ../Nutrition5K/train
  CSV path: ../Nutrition5K/nutrition5k_train.csv
  Output directory: ../experiments
  Batch size: 32
  Number of epochs: 40
  Image size: 256
  Workers: 4


In [13]:
# Create train/validation split
print("Creating train/validation split...")
train_csv, val_csv = create_train_val_split(
    CSV_PATH,
    val_ratio=VAL_RATIO,
    random_seed=42
)

print(f"Train CSV: {train_csv}")
print(f"Validation CSV: {val_csv}")

# Load a sample to check data
sample_dataset = Nutrition5KDataset(
    csv_path=train_csv,
    data_root=DATA_ROOT,
    split='train',
    augment=False,  # No augmentation for checking
    img_size=IMG_SIZE,
)

print(f"\nDataset loaded successfully!")
print(f"Training samples: {len(sample_dataset)}")
print(f"RGB shape: {sample_dataset[0]['rgb'].shape}")
print(f"Depth shape: {sample_dataset[0]['depth'].shape}")


Creating train/validation split...
Train CSV: ../Nutrition5K/train_split.csv
Validation CSV: ../Nutrition5K/val_split.csv
Loaded 2804 valid samples out of 2805

Dataset loaded successfully!
Training samples: 2804
RGB shape: torch.Size([3, 256, 256])
Depth shape: torch.Size([1, 256, 256])


# 3. Experiments

We'll conduct experiments to compare different encoder architectures with and without data augmentation.

**Architecture**: Dual-stream CNN with middle fusion
- **RGB encoder**: ResNet (18 or 34)
- **Depth encoder**: ResNet (18 or 34) 
- **Fusion**: Standard middle fusion (concatenate + 1x1 conv)

**Experiments**:
1. ResNet-18 without augmentation (baseline)
2. ResNet-18 with geometric augmentation
3. ResNet-34 without augmentation
4. ResNet-34 with geometric augmentation

## 3.1 Resnet-18

In [ ]:
### Define Hyperparameteres
DROPOUT_RATE = 0.3
FUSION_CHANNELS = 512
LEARNING_RATE = 8e-4
WEIGHT_DECAY = 1e-6
EARLY_STOPPING_PATIENCE = 7
WARMUP_RATIO = 0.1
MIN_LR_RATIO = 0.05

### 3.1.1 No Augmentation

In [15]:
#### Experiment: ResNet-18 without Data Augmentation

# Configuration for ResNet-18 baseline (no augmentation)
def train_resnet18_no_aug():
    """Train ResNet-18 with standard middle fusion, no data augmentation"""
    
    print("="*60)
    print("TRAINING: ResNet-18 + Middle Fusion (No Augmentation)")
    print("="*60)
    
    # Create datasets (no augmentation)
    train_dataset = Nutrition5KDataset(
        csv_path=train_csv,
        data_root=DATA_ROOT,
        split='train',
        augment=False,  # No augmentation
        img_size=IMG_SIZE,
    )
    
    val_dataset = Nutrition5KDataset(
        csv_path=val_csv,
        data_root=DATA_ROOT,
        split='val',
        augment=False,  # Never augment validation
        img_size=IMG_SIZE,
    )
    
    # Create data loaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=NUM_WORKERS,
        pin_memory=True if torch.cuda.is_available() else False,
        drop_last=True
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        num_workers=NUM_WORKERS,
        pin_memory=True if torch.cuda.is_available() else False
    )
    
    # Build model: ResNet-18 + Middle Fusion + Standard Regression Head
    model = build_model(
        encoder='resnet18',
        fusion='middle',
        regression_head='standard',
        pretrained=False,
        dropout_rate=DROPOUT_RATE,
        fusion_channels=FUSION_CHANNELS,
        use_segmentation=False  # Calorie prediction only
    )
    model = model.to(device)
    
    print(f"Model parameters: {model.get_num_parameters():,}")
    print(f"Training samples: {len(train_dataset)}")
    print(f"Validation samples: {len(val_dataset)}")
    
    # Loss function (calorie prediction only)
    criterion = nn.MSELoss()
    
    # Hyperparameters for this experiment
    learning_rate = LEARNING_RATE
    weight_decay = WEIGHT_DECAY
    
    # Optimizer
    optimizer = optim.AdamW(
        model.parameters(),
        lr=learning_rate,
        weight_decay=weight_decay
    )
    
    print(f"Learning rate: {learning_rate}")
    print(f"Weight decay: {weight_decay}")
    
    # Learning rate scheduler
    steps_per_epoch = len(train_loader)
    total_steps = NUM_EPOCHS * steps_per_epoch
    warmup_steps = int(total_steps * WARMUP_RATIO)

    # Learning rate scheduler: Warmup + Linear Decay
    scheduler = get_warmup_cosine_scheduler(
        optimizer, 
        warmup_steps=warmup_steps, 
        total_steps=total_steps,
        min_lr_ratio=MIN_LR_RATIO
    )
        
    # Create experiment directory
    exp_name = f"exp1_resnet18_no_aug_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    exp_dir = os.path.join(OUTPUT_DIR, exp_name)
    os.makedirs(exp_dir, exist_ok=True)
    
    # Create trainer
    trainer = Trainer(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        criterion=criterion,
        optimizer=optimizer,
        scheduler=scheduler,
        device=device,
        output_dir=exp_dir,
        early_stopping_patience=EARLY_STOPPING_PATIENCE,
        scheduler_step_on_batch=False
    )
    
    # Train the model
    trainer.train(NUM_EPOCHS)
    
    print(f"\nExperiment completed! Results saved to: {exp_dir}")
    return trainer.best_metrics

# Run the experiment
resnet18_no_aug_results = train_resnet18_no_aug()

TRAINING: ResNet-18 + Middle Fusion (No Augmentation)
Loaded 2804 valid samples out of 2805
Loaded 495 valid samples out of 495
Model parameters: 22,872,577
Training samples: 2804
Validation samples: 495
Learning rate: 0.0008
Weight decay: 1e-06
Starting training for 40 epochs...

Epoch 1/40


Validation: 100%|██████████| 16/16 [00:02<00:00,  7.52it/s]


Train Loss: 99588.1862
Val Loss: 107512.8083
MAE: 240.83

Epoch 2/40


Validation: 100%|██████████| 16/16 [00:02<00:00,  7.26it/s]


Train Loss: 93442.4327
Val Loss: 103962.4995
MAE: 237.11

Epoch 3/40


...

Train Loss: 2215.4360
Val Loss: 10430.9554
MAE: 64.50

Epoch 39/40


Validation: 100%|██████████| 16/16 [00:02<00:00,  7.36it/s]


Train Loss: 2082.9309
Val Loss: 9938.9886
MAE: 63.33

Epoch 40/40


Validation: 100%|██████████| 16/16 [00:02<00:00,  7.17it/s]

Train Loss: 2364.3041
Val Loss: 9860.9779
MAE: 63.48

Training completed!
Best validation loss: 9509.0767

Experiment completed! Results saved to: ../experiments/exp1_resnet18_no_aug_20251023_115441


### 3.1.2 Augmentation

In [17]:
#### Experiment: ResNet-18 with Data Augmentation

# Configuration for ResNet-18 with geometric augmentation
def train_resnet18_with_aug():
    """Train ResNet-18 with standard middle fusion and geometric data augmentation"""
    
    print("="*60)
    print("TRAINING: ResNet-18 + Middle Fusion (With Augmentation)")
    print("="*60)
    
    # Create datasets (with augmentation for training)
    train_dataset = Nutrition5KDataset(
        csv_path=train_csv,
        data_root=DATA_ROOT,
        split='train',
        augment=True,  # Enable geometric augmentation
        img_size=IMG_SIZE,
    )
    
    val_dataset = Nutrition5KDataset(
        csv_path=val_csv,
        data_root=DATA_ROOT,
        split='val',
        augment=False,  # Never augment validation
        img_size=IMG_SIZE,
    )
    
    # Create data loaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=NUM_WORKERS,
        pin_memory=True if torch.cuda.is_available() else False,
        drop_last=True
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        num_workers=NUM_WORKERS,
        pin_memory=True if torch.cuda.is_available() else False
    )
    
    # Build model: ResNet-18 + Middle Fusion + Standard Regression Head
    model = build_model(
        encoder='resnet18',
        fusion='middle',
        regression_head='standard',
        pretrained=False,
        dropout_rate=DROPOUT_RATE,
        fusion_channels=FUSION_CHANNELS,
        use_segmentation=False  # Calorie prediction only
    )
    model = model.to(device)
    
    print(f"Model parameters: {model.get_num_parameters():,}")
    print(f"Training samples: {len(train_dataset)}")
    print(f"Validation samples: {len(val_dataset)}")
    
    # Loss function (calorie prediction only)
    criterion = nn.MSELoss()
    
    # Hyperparameters for this experiment
    learning_rate = LEARNING_RATE
    weight_decay = WEIGHT_DECAY
    
    # Optimizer
    optimizer = optim.AdamW(
        model.parameters(),
        lr=learning_rate,
        weight_decay=weight_decay
    )
    
    print(f"Learning rate: {learning_rate}")
    print(f"Weight decay: {weight_decay}")
    
    # Learning rate scheduler
    steps_per_epoch = len(train_loader)
    total_steps = NUM_EPOCHS * steps_per_epoch
    warmup_steps = int(total_steps * WARMUP_RATIO)

    # Learning rate scheduler: Warmup + Linear Decay
    scheduler = get_warmup_cosine_scheduler(
        optimizer, 
        warmup_steps=warmup_steps, 
        total_steps=total_steps,
        min_lr_ratio=MIN_LR_RATIO
    )
    
    # Create experiment directory
    exp_name = f"exp2_resnet18_with_aug_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    exp_dir = os.path.join(OUTPUT_DIR, exp_name)
    os.makedirs(exp_dir, exist_ok=True)
    
    # Create trainer
    trainer = Trainer(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        criterion=criterion,
        optimizer=optimizer,
        scheduler=scheduler,
        device=device,
        output_dir=exp_dir,
        early_stopping_patience=EARLY_STOPPING_PATIENCE,
        scheduler_step_on_batch=False
    )
    
    # Train the model
    trainer.train(NUM_EPOCHS)
    
    print(f"\nExperiment completed! Results saved to: {exp_dir}")
    return trainer.best_metrics

# Run the experiment
resnet18_with_aug_results = train_resnet18_with_aug()

TRAINING: ResNet-18 + Middle Fusion (With Augmentation)
Loaded 2804 valid samples out of 2805
Loaded 495 valid samples out of 495
Model parameters: 22,872,577
Training samples: 2804
Validation samples: 495
Learning rate: 0.0008
Weight decay: 1e-06
Starting training for 40 epochs...

Epoch 1/40


Validation: 100%|██████████| 16/16 [00:02<00:00,  6.99it/s]


Train Loss: 99051.7583
Val Loss: 107375.8743
MAE: 240.59

Epoch 2/40


Validation: 100%|██████████| 16/16 [00:02<00:00,  7.29it/s]


Train Loss: 92092.3763
Val Loss: 89712.8545
MAE: 217.80

Epoch 3/40


...

Train Loss: 12543.2637
Val Loss: 14072.2663
MAE: 81.51

Epoch 29/40


Validation: 100%|██████████| 16/16 [00:02<00:00,  7.19it/s]


Train Loss: 11841.4046
Val Loss: 11159.2817
MAE: 72.23

Epoch 30/40


Validation: 100%|██████████| 16/16 [00:02<00:00,  7.19it/s]

Train Loss: 10617.0792
Val Loss: 11197.8942
MAE: 70.36
Early stopping triggered after 30 epochs
Best epoch: 23

Training completed!
Best validation loss: 10907.9683

Experiment completed! Results saved to: ../experiments/exp2_resnet18_with_aug_20251023_120817


## 3.2 Resnet-34

In [23]:
DROPOUT_RATE = 0.4

### 3.2.1 No Augmentation

In [24]:
#### Experiment: ResNet-34 without Data Augmentation

# Configuration for ResNet-34 baseline (no augmentation)
def train_resnet34_no_aug():
    """Train ResNet-34 with standard middle fusion, no data augmentation"""
    
    print("="*60)
    print("TRAINING: ResNet-34 + Middle Fusion (No Augmentation)")
    print("="*60)
    
    # Create datasets (no augmentation)
    train_dataset = Nutrition5KDataset(
        csv_path=train_csv,
        data_root=DATA_ROOT,
        split='train',
        augment=False,  # No augmentation
        img_size=IMG_SIZE,
    )
    
    val_dataset = Nutrition5KDataset(
        csv_path=val_csv,
        data_root=DATA_ROOT,
        split='val',
        augment=False,  # Never augment validation
        img_size=IMG_SIZE,
    )
    
    # Create data loaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=NUM_WORKERS,
        pin_memory=True if torch.cuda.is_available() else False,
        drop_last=True
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        num_workers=NUM_WORKERS,
        pin_memory=True if torch.cuda.is_available() else False
    )
    
    # Build model: ResNet-34 + Middle Fusion + Standard Regression Head
    model = build_model(
        encoder='resnet34',  # ResNet-34 instead of ResNet-18
        fusion='middle',
        regression_head='standard',
        pretrained=False,
        dropout_rate=DROPOUT_RATE,
        fusion_channels=FUSION_CHANNELS,
    )
    model = model.to(device)
    
    print(f"Model parameters: {model.get_num_parameters():,}")
    print(f"Training samples: {len(train_dataset)}")
    print(f"Validation samples: {len(val_dataset)}")
    
    # Loss function (calorie prediction only)
    criterion = nn.MSELoss()
    
    # Hyperparameters for this experiment
    learning_rate = LEARNING_RATE
    weight_decay = WEIGHT_DECAY
    
    # Optimizer
    optimizer = optim.AdamW(
        model.parameters(),
        lr=learning_rate,
        weight_decay=weight_decay
    )
    
    print(f"Learning rate: {learning_rate}")
    print(f"Weight decay: {weight_decay}")
    
    # Learning rate scheduler
    steps_per_epoch = len(train_loader)
    total_steps = NUM_EPOCHS * steps_per_epoch
    warmup_steps = int(total_steps * WARMUP_RATIO)

    # Learning rate scheduler: Warmup + Linear Decay
    scheduler = get_warmup_cosine_scheduler(
        optimizer, 
        warmup_steps=warmup_steps, 
        total_steps=total_steps,
        min_lr_ratio=MIN_LR_RATIO
    )
    
    # Create experiment directory
    exp_name = f"exp3_resnet34_no_aug_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    exp_dir = os.path.join(OUTPUT_DIR, exp_name)
    os.makedirs(exp_dir, exist_ok=True)
    
    # Create trainer
    trainer = Trainer(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        criterion=criterion,
        optimizer=optimizer,
        scheduler=scheduler,
        device=device,
        output_dir=exp_dir,
        early_stopping_patience=EARLY_STOPPING_PATIENCE,
        scheduler_step_on_batch=False
    )
    
    # Train the model
    trainer.train(NUM_EPOCHS)
    
    print(f"\nExperiment completed! Results saved to: {exp_dir}")
    return trainer.best_metrics

# Run the experiment
resnet34_no_aug_results = train_resnet34_no_aug()

TRAINING: ResNet-34 + Middle Fusion (No Augmentation)
Loaded 2804 valid samples out of 2805
Loaded 495 valid samples out of 495
Model parameters: 43,088,897
Training samples: 2804
Validation samples: 495
Learning rate: 0.0008
Weight decay: 1e-06
Starting training for 40 epochs...

Epoch 1/40


Validation: 100%|██████████| 16/16 [00:02<00:00,  7.24it/s]


Train Loss: 98698.4380
Val Loss: 107480.8511
MAE: 240.78

Epoch 2/40


Validation: 100%|██████████| 16/16 [00:02<00:00,  7.31it/s]


Train Loss: 93298.1988
Val Loss: 94860.5649
MAE: 225.05

Epoch 3/40


...

Train Loss: 4288.3274
Val Loss: 9847.6848
MAE: 66.14

Epoch 39/40


Validation: 100%|██████████| 16/16 [00:02<00:00,  7.38it/s]


Train Loss: 4522.8226
Val Loss: 9699.7839
MAE: 65.64

Epoch 40/40


Validation: 100%|██████████| 16/16 [00:02<00:00,  7.13it/s]

Train Loss: 4215.9819
Val Loss: 9186.5228
MAE: 66.61
Early stopping triggered after 40 epochs
Best epoch: 33

Training completed!
Best validation loss: 9133.4005

Experiment completed! Results saved to: ../experiments/exp3_resnet34_no_aug_20251023_122720


### 3.2.2 Augmentation

In [ ]:
#### Experiment: ResNet-34 with Data Augmentation

# Configuration for ResNet-34 with geometric augmentation
def train_resnet34_with_aug():
    """Train ResNet-34 with standard middle fusion and geometric data augmentation"""
    
    print("="*60)
    print("TRAINING: ResNet-34 + Middle Fusion (With Augmentation)")
    print("="*60)
    
    # Create datasets (with augmentation for training)
    train_dataset = Nutrition5KDataset(
        csv_path=train_csv,
        data_root=DATA_ROOT,
        split='train',
        augment=True,  # Enable geometric augmentation
        img_size=IMG_SIZE,
    )
    
    val_dataset = Nutrition5KDataset(
        csv_path=val_csv,
        data_root=DATA_ROOT,
        split='val',
        augment=False,  # Never augment validation
        img_size=IMG_SIZE,
    )
    
    # Create data loaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=NUM_WORKERS,
        pin_memory=True if torch.cuda.is_available() else False,
        drop_last=True
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        num_workers=NUM_WORKERS,
        pin_memory=True if torch.cuda.is_available() else False
    )
    
    # Build model: ResNet-34 + Middle Fusion + Standard Regression Head
    model = build_model(
        encoder='resnet34',  # ResNet-34 instead of ResNet-18
        fusion='middle',
        regression_head='standard',
        pretrained=False,
        dropout_rate=DROPOUT_RATE,
        fusion_channels=FUSION_CHANNELS,
    )
    model = model.to(device)
    
    print(f"Model parameters: {model.get_num_parameters():,}")
    print(f"Training samples: {len(train_dataset)}")
    print(f"Validation samples: {len(val_dataset)}")
    
    # Loss function (calorie prediction only)
    criterion = nn.MSELoss()
    
    # Hyperparameters for this experiment
    learning_rate = LEARNING_RATE
    weight_decay = WEIGHT_DECAY
    
    # Optimizer
    optimizer = optim.AdamW(
        model.parameters(),
        lr=learning_rate,
        weight_decay=weight_decay
    )
    
    print(f"Learning rate: {learning_rate}")
    print(f"Weight decay: {weight_decay}")
    
    steps_per_epoch = len(train_loader)
    total_steps = NUM_EPOCHS * steps_per_epoch
    warmup_steps = int(total_steps * WARMUP_RATIO)

    # Learning rate scheduler: Warmup + Linear Decay
    scheduler = get_warmup_cosine_scheduler(
        optimizer, 
        warmup_steps=warmup_steps, 
        total_steps=total_steps,
        min_lr_ratio=MIN_LR_RATIO
    )
    
    # Create experiment directory
    exp_name = f"exp4_resnet34_with_aug_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    exp_dir = os.path.join(OUTPUT_DIR, exp_name)
    os.makedirs(exp_dir, exist_ok=True)
    
    # Create trainer
    trainer = Trainer(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        criterion=criterion,
        optimizer=optimizer,
        scheduler=scheduler,
        device=device,
        output_dir=exp_dir,
        early_stopping_patience=EARLY_STOPPING_PATIENCE,
        scheduler_step_on_batch=False
    )
    
    # Train the model
    trainer.train(NUM_EPOCHS)
    
    print(f"\nExperiment completed! Results saved to: {exp_dir}")
    return trainer.best_metrics

# Run the experiment
resnet34_with_aug_results = train_resnet34_with_aug()

TRAINING: ResNet-34 + Middle Fusion (With Augmentation)
Loaded 2804 valid samples out of 2805
Loaded 495 valid samples out of 495
Model parameters: 43,088,897
Training samples: 2804
Validation samples: 495
Learning rate: 0.0008
Weight decay: 1e-06
Starting training for 40 epochs...

Epoch 1/40


Validation: 100%|██████████| 16/16 [00:02<00:00,  7.31it/s]


Train Loss: 99124.6188
Val Loss: 107322.5024
MAE: 240.47

Epoch 2/40


Validation: 100%|██████████| 16/16 [00:02<00:00,  7.15it/s]


Train Loss: 92163.1452
Val Loss: 70563.1094
MAE: 189.04

Epoch 3/40


...

Train Loss: 11471.6048
Val Loss: 22065.4567
MAE: 112.77

Epoch 34/40


Validation: 100%|██████████| 16/16 [00:02<00:00,  7.08it/s]


Train Loss: 10118.9804
Val Loss: 12113.4151
MAE: 74.36

Epoch 35/40


Validation: 100%|██████████| 16/16 [00:02<00:00,  7.31it/s]

Train Loss: 10894.2290
Val Loss: 30447.4351
MAE: 115.17
Early stopping triggered after 35 epochs
Best epoch: 28

Training completed!
Best validation loss: 11760.0813

Experiment completed! Results saved to: ../experiments/exp4_resnet34_with_aug_20251023_123711


## Results Summary and Analysis

Compare the results from different encoder architectures and the effect of data augmentation.


In [28]:
# Compare results from all encoder experiments
def compare_encoder_results():
    """Compare results from all encoder experiments"""
    
    print("="*80)
    print("ENCODER EXPERIMENT RESULTS COMPARISON")
    print("="*80)
    
    # Collect results (these variables should exist after running experiments)
    results = [
        ("ResNet-18 (No Aug)", resnet18_no_aug_results),
        ("ResNet-18 (With Aug)", resnet18_with_aug_results),
        ("ResNet-34 (No Aug)", resnet34_no_aug_results),
        ("ResNet-34 (With Aug)", resnet34_with_aug_results)
    ]
    
    # Display results in a table format
    print(f"{'Experiment':<25} {'Val Loss':<10} {'MAE':<10} {'Best Epoch':<12}")
    print("-" * 80)
    
    for name, metrics in results:
        val_loss = metrics['val_loss']
        mae = metrics['mae']
        epoch = metrics['epoch']
        
        print(f"{name:<25} {val_loss:<10.4f} {mae:<10.2f} {epoch:<12}")
    
    print("\n" + "="*80)
    
# Run the comparison
compare_encoder_results()


ENCODER EXPERIMENT RESULTS COMPARISON
Experiment                Val Loss   MAE        Best Epoch  
--------------------------------------------------------------------------------
ResNet-18 (No Aug)        9509.0767  63.78      35          
ResNet-18 (With Aug)      10907.9683 73.49      23          
ResNet-34 (No Aug)        9133.4005  66.62      33          
ResNet-34 (With Aug)      11760.0813 74.16      28          

